In [ ]:
import time
import os
import random
import numpy as np
import torch
import torch.utils.data
from PIL import Image
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data.dataloader import DataLoader
import numpy as np
import torch
from torch.utils.data.dataset import Dataset
import matplotlib.pylab as plt


from PIL import Image
from torchvision import transforms
mypath = "C:/Users/sunyu666/Desktop/cirGAN/data/my308470"
from model import Generator
from model import Discriminator

In [ ]:
def show_pics(pics,heatmap = np.zeros((1,1))):
    plt.figure(figsize=(3*len(pics),3),dpi=80)
    for i in range(len(pics)):
        pics[i] = (pics[i][0].transpose((1,2,0))+1) / 2
        plt.subplot(1,len(pics),i+1)
        print(pics[i])
        plt.imshow(pics[i])
        plt.xticks([])
        plt.yticks([])  

In [ ]:
transforms_1 = [
    transforms.Resize(int(256 * 1.12)),   ## 图片放大1.12倍
    transforms.RandomCrop((256, 256)),         ## 随机裁剪成原来的大小
    transforms.RandomHorizontalFlip(),                              ## 随机水平翻转
    transforms.ToTensor(),                                          ## 变为Tensor数据
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),         ## 正则化
    ]

In [ ]:
class CFG():
    def __init__(self):
        self.data_dir='./data/my308470/'
        self.shuffle = True
        self.dataset = mypath
        self.model_net = 'CycleGAN'
        self.train_list = None
        self.batch_size = 1
        self.drop_last = False
        self.run_test = True
        self.crop_size = 224
        self.load_size = 256
        self.crop_type = 'Random'
        self.phase = 'train'
        self.list_filename = None
        self.use_gpu = True
        self.return_name = False



class reader_creator(Dataset):
    def __init__(self,this_cfg,transforms_ = None):
        super(Dataset,self).__init__()
        self.image_dir =  this_cfg.image_dir
        self.shuffle = this_cfg.shuffle
        self.dataset = this_cfg.dataset
        self.model_net = this_cfg.model_net
        self.list_filename = this_cfg.list_filename
        self.batch_size = this_cfg.batch_size
        self.drop_last = this_cfg.drop_last
        self.run_test = this_cfg.run_test
        self.load_size = this_cfg.load_size
        self.crop_type = this_cfg.crop_type
        self.crop_size = this_cfg.crop_size
        self.phase = this_cfg.phase
        self.return_name = this_cfg.return_name
        self.lines = open(self.list_filename).readlines()
        self.cfg = this_cfg
        self.transform = transforms.Compose(transforms_) 
        if self.shuffle:
            np.random.shuffle(self.lines)

    def len(self):
        if self.drop_last or len(self.lines) % self.batch_size == 0:
            return len(self.lines) // self.batch_size
        else:
            return len(self.lines) // self.batch_size + 1
    def __len__(self):
        return len(self.lines)

    def __getitem__(self, index):
        file = self.lines[index]
        file = file.strip('\n\t\r')
        img  = Image.open(os.path.join(self.image_dir,file)).convert('RGB')
        img = self.transform(img)
        return img



class dataloader(Dataset):
    def __init__(self,cfg):
        super(Dataset,self).__init__()
        self.cfg = cfg
        self.shuffle = self.cfg.shuffle
    def make_data(self):
        data_dir = os.path.join(self.cfg.data_dir)
        trainA_list = os.path.join(data_dir,'trainA.txt')
        trainB_list = os.path.join(data_dir,'trainB.txt')
        self.cfg.image_dir = data_dir
        self.cfg.list_filename = trainA_list
        self.cfg.phase = 'train'
        a_train_reader = reader_creator(self.cfg,transforms_1)
        self.cfg.list_filename = trainB_list
        b_train_reader = reader_creator(self.cfg,transforms_1)
        a_test_reader = None
        b_test_reader = None
        batch_num = max(a_train_reader.len(),b_train_reader.len())
        return a_train_reader,b_train_reader,a_test_reader,b_test_reader,batch_num

#cfg = CFG()

#reader = dataloader(cfg)
#a_reader,b_reader,a_test,b_test,batch_num = reader.make_data()

#A_reader = DataLoader(a_reader,shuffle=True,drop_last=cfg.drop_last,batch_size=cfg.batch_size,num_workers=0)
#B_reader = DataLoader(b_reader,shuffle=True,drop_last=cfg.drop_last,batch_size=cfg.batch_size,num_workers=0)


#data_A = A_reader
'''
print(len(data_A))
for  data in zip(data_A):
    print(data.shape[0],data.shape[1],data.shape[2],data.shape[3])
    print(len(data))
'''

In [ ]:
def save_pic(pics,file_name = 'temp',save_path = './output/pics/'):
    for i in range(len(pics)):
        pics[i] = pics[i][0]
    pic = np.concatenate(tuple(pics),axis=2)
    pic = pic.transpose((1,2,0))
    pic = np.clip(pic*256,0,255)
    img = Image.fromarray(pic.astype('uint8')).convert('RGB')
    img.save(save_path+file_name+'.jpg')

In [ ]:
print(torch.cuda.is_available())

In [ ]:
import time
from PIL import Image, ImageEnhance
class ImagePool(object):
    def __init__(self,pool_size = 50):
        self.pool = []
        self.count = 0
        self.pool_size = pool_size
    def pool_image(self,image):
        rnt = image
        if self.count < self.pool_size:
            self.pool.append(image)
            self.count += 1
            rnt = image
        else:
            p = np.random.rand()
            print(p)
            if p > 0.5:
                random_id = np.random.randint(0,self.pool_size-1)
                temp = self.pool[random_id]
                #print(temp)
                self.pool[random_id] = image
                rtn = temp
            else:
                rtn = image
                #print(rnt)
        return (rnt)



In [ ]:
def _save_model(self,epoch):
    print('save model epoch %d'%epoch)
    torch.save
def trian(epoch_num = 1000,
            adv_weight = 1,
        cycle_weight = 30,
        identity_weight = 10,
        use_gpu = True,
        load_model = False,
        model_path = './model/',
        model_path_bkp = './model_bkp/',
        print_interval = 1,
        max_step = 50,
        model_bkp_intercval = 5000):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    
    g_x = Generator().to(device)
    g_y = Generator().to(device)

    #此处创建两个判别器
    d_x = Discriminator().to(device)
    d_y = Discriminator().to(device)

    cfg = CFG()
    loader = dataloader(cfg)
    a_reader,b_reader,a_test_reader,b_test_reader,batch_num = loader.make_data()
    
    x_train_data = DataLoader(a_reader,shuffle=True,drop_last=cfg.drop_last,batch_size=cfg.batch_size,num_workers=0)
    y_train_data = DataLoader(b_reader,shuffle=True,drop_last=cfg.drop_last,batch_size=cfg.batch_size,num_workers=0)

    

    g_x_optimizer = torch.optim.Adam(g_x.parameters(),lr= 0.0001,betas=(0.5,0.999))
    g_y_optimizer = torch.optim.Adam(g_y.parameters(),lr= 0.0001,betas=(0.5,0.999))

    d_x_optimizer = torch.optim.Adam(d_x.parameters(),lr= 0.0001,betas=(0.5,0.999))
    d_y_optimizer = torch.optim.Adam(d_y.parameters(),lr= 0.0001,betas=(0.5,0.999))

    fb_pool = ImagePool()
    fa_pool = ImagePool()
    
    #if load_model == True:#load_model
    steps = 0
    
    for epoch in range(100):
        print("----epoches%d -------"%(epoch))
        for data_x,data_y in zip(x_train_data,y_train_data):
            steps += 1
            #print(len(data_x))
            #print(len(data_y))
            print(steps)
            image_rx = data_x[0].reshape(1,3,256,256).to(device)
            image_ry = data_y[0].reshape(1,3,256,256).to(device)
            
            #print(type(image_rx))
            #print(image_rx.shape)


            gx_gan_loss = torch.mean((d_x(g_x(image_ry.detach()))-1)**2)
            gx_cycle_loss = torch.mean(torch.abs(image_ry.detach() - g_y(g_x(image_ry.detach()))))
            gx_ide_loss = torch.mean(torch.abs(image_rx.detach() - (g_x(image_rx.detach()))))
            gx_loss = adv_weight * gx_gan_loss + cycle_weight * gx_cycle_loss + identity_weight * gx_ide_loss
            g_x_optimizer.zero_grad()
            gx_loss.backward()
            g_x_optimizer.step()
            #训练生成器1

            #print(gx_loss)

            gy_gan_loss = torch.mean((d_y(g_y(image_rx.detach()))-1)**2)
            gy_cycle_loss = torch.mean(torch.abs(image_rx.detach() - g_x(g_y(image_rx.detach()))))
            gy_ide_loss = torch.mean(torch.abs(image_ry.detach() - (g_y(image_ry.detach()))))
            gy_loss = adv_weight * gy_gan_loss + cycle_weight * gy_cycle_loss + identity_weight * gy_ide_loss
            g_y_optimizer.zero_grad()
            gy_loss.backward()
            g_y_optimizer.step()
            #训练生成器2
            #print(gy_loss)
            #print("------------------")

            d_loss_rx = torch.mean((d_x(image_rx.detach())-1)**2) 
            #print(d_loss_ra)
            d_loss_fx = torch.mean(d_x(fa_pool.pool_image(g_x(image_ry.detach())))**2)
            #print(d_loss_fa)
            d_x_loss = (d_loss_fx + d_loss_rx) * 0.5
            d_x_optimizer.zero_grad()
            d_x_loss.backward()
            d_x_optimizer.step()
            #print(d_x_loss)
            #训练判别器1


            d_loss_ry = torch.mean((d_y(image_ry.detach())-1)**2) 
            d_loss_fy = torch.mean(d_y(fb_pool.pool_image(g_y(image_rx.detach())))**2)
            d_y_loss = (d_loss_fy + d_loss_ry) * 0.5
            d_y_optimizer.zero_grad()
            d_y_loss.backward()
            d_y_optimizer.step()
            #训练判别器2
            #print(d_y_loss)

            #print(image_ry.detach().shape[0],image_ry.detach().shape[1],image_ry.detach().shape[2])
            #print((g_x(image_ry.detach())).shape[0],(g_x(image_ry.detach())).shape[1],(g_x(image_ry.detach())).shape[2])
            #print(g_y(g_x(image_ry.detach())).shape[0],g_y(g_x(image_ry.detach())).shape[1],g_y(g_x(image_ry.detach())).shape[2])
            
        if(epoch == 1):
            print('save model epoch %d'%epoch)
            torch.save(g_x.state_dict(),model_path+'gy2x%d.pth'%epoch)
            torch.save(d_x.state_dict(),model_path+'dx%d.pth'%epoch)
            torch.save(g_y.state_dict(),model_path+'gx2y%d.pth'%epoch)
            torch.save(d_y.state_dict(),model_path+'dy%d.pth'%epoch)
        if (epoch%5 == 0):
            print('save model epoch %d'%epoch)
            torch.save(g_x.state_dict(),model_path+'gy2x%d.pth'%epoch)
            torch.save(d_x.state_dict(),model_path+'dx%d.pth'%epoch)
            torch.save(g_y.state_dict(),model_path+'gx2y%d.pth'%epoch)
            torch.save(d_y.state_dict(),model_path+'dy%d.pth'%epoch)
            
            
trian()
        

In [ ]:
from torchvision.utils import save_image
from torch.autograd import Variable
from models import GeneratorResNet
from models import Discriminator

In [ ]:
def infer(max_step = 10,use_gpu = True,load_model = True,model_path = './model/'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #g_a = GeneratorResNet((3,256,256),9)
    g_a = Generator()
    g_b = Generator()
    cfg  = CFG()
    g_a.load_state_dict(torch.load('./model/gx2y_5.pth'))
    img  = Image.open(os.path.join('./data/my308470/trainA/female_10.jpg')).convert('RGB')
    
    g_b.load_state_dict(torch.load('./model/gy2x_5.pth'))
    img2  = Image.open(os.path.join('./data/my308470/testB/0001.jpg')).convert('RGB')
    #i2 = Variable(img)
    t  = transforms.Compose(transforms_1) 
    i = t(img)
    i = i.resize(1,3,256,256)

    t  = transforms.Compose(transforms_1) 
    i2 = t(img2)
    i2 = i2.resize(1,3,256,256)
    #img = img.resize((256,256),Image.BICUBIC)    
    #img = np.array(img)
    #print(img)
    #plt.imshow(i)
    #img = torch.tensor(img)
    #img = img.reshape(1,3,256,256).float()
    #print(img.shape)
    img_2 =  0.5*(g_a(i).data + 1.0)
    save_image(img_2,'./data/output1.jpg')
    img_3 =  0.5*(g_b(i2).data + 1.0)
    save_image(img_3,'./data/output2.jpg')
    return
infer()